# Database Connection
### =======================================================

In [5]:
import oracledb
import pandas as pd
import os

In [6]:

username = "myuser"
password = "Mikewon98"
dsn = "DESKTOP-LNSO41D:1521/XEPDB1"

csv_path = '../data/.combined_clean_bank_reviews.csv'
try:
    df = pd.read_csv(csv_path)
    df['date'] = pd.to_datetime(df['date'])
    print(f"✅ Successfully loaded {len(df)} reviews from CSV.")
except FileNotFoundError:
    print(f"❌ ERROR: The file was not found at {csv_path}. Please check the path.")
    exit()


connection = None
try:
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    cursor = connection.cursor()
    print("✅ Database connection successful.")

    bank_lookup = {}
    unique_banks = df['bank'].unique()
    
    print("\nPopulating Banks table with unique banks...")
    for bank_name in unique_banks:
        # Check if bank already exists
        cursor.execute("SELECT bank_id FROM Banks WHERE bank_name = :1", [bank_name])
        result = cursor.fetchone()

        if result:
            # Bank already exists, use its ID
            bank_id = result[0]
        else:
            # Bank is new, insert it and get the new ID back

            # 1. Create a special variable to receive the output from the database
            out_var = cursor.var(oracledb.DB_TYPE_NUMBER)

            # 2. Execute the INSERT, telling Oracle to place the new ID INTO our variable
            cursor.execute("""
                INSERT INTO Banks (bank_name) 
                VALUES (:1) 
                RETURNING bank_id INTO :2
            """, [bank_name, out_var])

            # 3. Get the value FROM THE VARIABLE (out_var), not the cursor. This is the fix.
            bank_id = out_var.getvalue()[0]

        # Add the bank name and its ID to our lookup dictionary for later use
        bank_lookup[bank_name] = bank_id

    print(f"✅ Banks table populated. Found {len(bank_lookup)} unique banks.")
    
    # --- B. Prepare and Insert Reviews in a Batch ---
    print("\nPreparing review data for batch insert...")
    
    # Prepare data for executemany(), which is much faster than one-by-one inserts
    reviews_to_insert = []
    for index, row in df.iterrows():
        bank_id = bank_lookup[row['bank']] # Get foreign key from our lookup dictionary
        # Append a tuple in the correct order for the SQL statement
        reviews_to_insert.append(
            (bank_id, row['review'], row['rating'], row['date'], row['source'])
        )

    print(f"🚀 Inserting {len(reviews_to_insert)} reviews into the database...")
    
    # Use executemany() for efficient batch processing
    cursor.executemany("""
        INSERT INTO Reviews (bank_id, review_text, rating, review_date, source)
        VALUES (:1, :2, :3, :4, :5)
    """, reviews_to_insert)

    # Commit the transaction to make the changes permanent
    connection.commit()
    
    print(f"🎉 Success! {cursor.rowcount} reviews have been inserted and committed to the database.")

except oracledb.DatabaseError as e:
    print(f"❌ Database error: {e}")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

finally:
    # --- 4. Clean Up ---
    if connection:
        cursor.close()
        connection.close()
        print("\nDatabase connection closed.")

✅ Successfully loaded 3478 reviews from CSV.
✅ Database connection successful.

Populating Banks table with unique banks...
✅ Banks table populated. Found 3 unique banks.

Preparing review data for batch insert...
🚀 Inserting 3478 reviews into the database...
🎉 Success! 3478 reviews have been inserted and committed to the database.

Database connection closed.
